---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

In [3]:
def answer_one():
    
    return spam_data['target'].sum()/spam_data['target'].size

In [4]:
answer_one()

0.13406317300789664

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    vect = CountVectorizer().fit(X_train)
    tokens = vect.get_feature_names()
    lengths = [len(t) for t in tokens]
    max_index = lengths.index(max(lengths))
    
    return tokens[max_index]

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    clfrNB = MultinomialNB(alpha=0.1)
    clfrNB.fit(X_train_vectorized,y_train)
    predictions = clfrNB.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

In [8]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    feature_names = vect.get_feature_names()
    tfidf = vect.idf_
    
    res = pd.DataFrame(data=list(zip(feature_names,tfidf)),columns=['feature_name','tfidf_value'])
    res_asc = res.sort_values(['tfidf_value', 'feature_name'], ascending=[True, True])[:20]
    res_desc = res.sort_values(['tfidf_value', 'feature_name'], ascending=[False, True])[:20]
    
    res_asc.set_index('feature_name',inplace=True)
    res_desc.set_index('feature_name',inplace=True)
    
    return (res_asc,res_desc)

In [10]:
answer_four()

(              tfidf_value
 feature_name             
 to               2.198406
 you              2.265645
 the              2.707383
 in               2.890761
 and              2.976764
 is               3.003012
 me               3.111530
 for              3.206840
 it               3.222174
 my               3.231044
 call             3.297812
 your             3.300196
 of               3.319473
 have             3.354130
 that             3.408477
 on               3.463136
 now              3.465949
 can              3.545053
 are              3.560414
 so               3.566625,               tfidf_value
 feature_name             
 000pes           8.644919
 0089             8.644919
 0121             8.644919
 01223585236      8.644919
 0125698789       8.644919
 02072069400      8.644919
 02073162414      8.644919
 02085076972      8.644919
 021              8.644919
 0430             8.644919
 07008009200      8.644919
 07099833605      8.644919
 07123456789      8.644919
 

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
def answer_five():
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    clfrNB = MultinomialNB(alpha=0.1)
    clfrNB.fit(X_train_vectorized,y_train)
    predictions = clfrNB.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

In [12]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
    
    non_spam = spam_data[spam_data['target']==0]
    spam = spam_data[spam_data['target']==1]
    
    len_ns = [len(t) for t in non_spam['text']]
    len_s = [len(t) for t in spam['text']]
    
    avg_len_ns = sum(len_ns)/len(len_ns)
    avg_len_s = sum(len_s)/len(len_s)
    
    return (avg_len_ns,avg_len_s)

In [14]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC

def answer_seven():
    
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    #Compute doc length and add feature
    doc_len = [len(t) for t in X_train]
    X_train_added = add_feature(X_train_vectorized, doc_len)
    
    #Fit SVC
    clfrSVM = SVC(C=10000)
    clfrSVM.fit(X_train_added,y_train)
    
    #Report roc
    X_test_vectorized = vect.transform(X_test)
    doc_len2 = [len(t) for t in X_test]
    X_test_added = add_feature(X_test_vectorized, doc_len2)
    predictions = clfrSVM.predict(X_test_added)
    
    return roc_auc_score(y_test, predictions)

In [17]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
def answer_eight():
    
    count = lambda l1,l2: sum([1 for x in l1 if x in l2])
    
    non_spam = spam_data[spam_data['target']==0]
    spam = spam_data[spam_data['target']==1]
    
    num_digits_ns = [count(t,'0123456789') for t in non_spam['text']]
    num_digits_s = [count(t,'0123456789') for t in spam['text']]
    
    avg_nd_ns = sum(num_digits_ns)/len(num_digits_ns)
    avg_nd_s = sum(num_digits_s)/len(num_digits_s)
    
    return (avg_nd_ns,avg_nd_s)

In [19]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [20]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    vect = TfidfVectorizer(min_df=5,ngram_range=(1,3)).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    #Compute doc length and number of digits and add feature
    count = lambda l1,l2: sum([1 for x in l1 if x in l2])
    
    doc_len = [len(t) for t in X_train]
    number_digits = [count(t,'0123456789') for t in X_train]
    X_train_added = add_feature(X_train_vectorized, [doc_len,number_digits])
    
    #Fit logistic regression
    model = LogisticRegression(C=100)
    model.fit(X_train_added, y_train)
    
    #Report roc
    X_test_vectorized = vect.transform(X_test)
    doc_len2 = [len(t) for t in X_test]
    number_digits2 = [count(t,'0123456789') for t in X_test]
    X_test_added = add_feature(X_test_vectorized, [doc_len2,number_digits2])
    predictions = model.predict(X_test_added)
    
    return roc_auc_score(y_test, predictions)

In [21]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [22]:
def answer_ten():
    
    count = lambda l1,l2: sum([1 for x in l1 if not x in l2])
    
    non_spam = spam_data[spam_data['target']==0]
    spam = spam_data[spam_data['target']==1]
    
    nwc_ns = [count(t,'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789_') for t in non_spam['text']]
    nwc_s = [count(t,'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789_') for t in spam['text']]
    
    avg_nwc_ns = sum(nwc_ns)/len(nwc_ns)
    avg_nwc_s = sum(nwc_s)/len(nwc_s)
    
    return (avg_nwc_ns,avg_nwc_s)

In [23]:
answer_ten()

(17.37720207253886, 29.49665327978581)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [24]:
def answer_eleven():
    
    vect = CountVectorizer(min_df=5,ngram_range=(2,5),analyzer='char_wb').fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    #Compute doc length and number of digits and add feature
    count = lambda l1,l2: sum([1 for x in l1 if x in l2])
    count2 = lambda l1,l2: sum([1 for x in l1 if not x in l2])
    
    length_of_doc = [len(t) for t in X_train]
    digit_count = [count(t,'0123456789') for t in X_train]
    non_word_char_count = [count(t,'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789_') for t in X_train]
    X_train_added = add_feature(X_train_vectorized, [length_of_doc,digit_count,non_word_char_count])
    
    #Fit logistic regression
    model = LogisticRegression(C=100)
    model.fit(X_train_added, y_train)
    
    #Report roc
    X_test_vectorized = vect.transform(X_test)
    length_of_doc = [len(t) for t in X_test]
    digit_count = [count(t,'0123456789') for t in X_test]
    non_word_char_count = [count(t,'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789_') for t in X_test]
    X_test_added = add_feature(X_test_vectorized, [length_of_doc,digit_count,non_word_char_count])
    predictions = model.predict(X_test_added)
    
    #Feature names
    feature_names = vect.get_feature_names()
    feature_names.extend(['length_of_doc','digit_count','non_word_char_count'])
    features = np.array(feature_names)
    sorted_coef_index = model.coef_[0].argsort()
    small_coef = features[sorted_coef_index[:10]].tolist()
    large_coef = features[sorted_coef_index[:-11:-1]].tolist()
    
    return (roc_auc_score(y_test, predictions), small_coef, large_coef)

In [25]:
answer_eleven()

(0.97885931107074342,
 ['..', '. ', ' i', '? ', ' y', ' go', ':)', ' m', ' h', 'h '],
 ['digit_count', 'ne', 'co', 'ia', 'xt', 'mob', 'ww', ' ch', ' x', 'ar'])